# Stock Trading Bot Ayyyyy
- api reference: https://www.alphavantage.co/documentation/ and potentially https://site.financialmodelingprep.com/developer/docs

In [1]:
!pip install graphviz python-decouple

In [42]:
# This is just importing certain libraries we will use consistently throughouit the porject
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import requests

In [43]:
import config

api_key = config.api_key
base_url = "https://www.alphavantage.co/query?"

In [44]:
# ticker_set = set(("AMD"))
daily_ticker_function = "TIME_SERIES_DAILY"
symbol = "DDOG"
daily_ticker_url = f"{base_url}function={daily_ticker_function}&symbol={symbol}&apikey={api_key}"
daily_ticker_response = requests.get(daily_ticker_url)
daily_ticker_data = daily_ticker_response.json()

In [45]:
daily_data = pd.DataFrame.from_dict(daily_ticker_data["Time Series (Daily)"], orient="index")
daily_data.head()

,1. open,2. high,3. low,4. close,5. volume
2023-09-29,91.8400,93.2200,90.9500,91.0900,3797923
2023-09-28,86.9050,90.1100,85.2900,88.9600,4128735
2023-09-27,87.0600,88.3600,86.6600,87.8300,2256020
2023-09-26,87.9300,88.7500,86.2500,86.5700,2835372
2023-09-25,87.8400,88.8800,87.4010,88.8400,1665995


In [46]:
# # Feature engineering (you can add more relevant features)
daily_data['4. close'] = daily_data['4. close'].astype(float)
daily_data['1. open'] = daily_data['1. open'].astype(float)

In [47]:
print(data.dtypes)

1. open           float64
2. high            object
3. low             object
4. close          float64
5. volume           int32
Close_Prev_Day    float64
Open_Next_Day     float64
volume_change     float64
Price_Diff        float64
dtype: object


In [50]:
daily_data['Close_Prev_Day'] = daily_data['4. close'].shift(1)

# Then, create 'Open_Next_Day' by shifting 'Open' one day forward
daily_data['Open_Next_Day'] = daily_data['1. open'].shift(-1)

# Calculate the difference between 'Close_Prev_Day' and 'Open_Next_Day'
daily_data['Price_Diff'] = daily_data['Open_Next_Day'] - daily_data['Close_Prev_Day']

# Drop the NaN values in the new column, which will be present in the first and last rows
daily_data.dropna(subset=['Price_Diff'], inplace=True)
daily_data.head()

,1. open,2. high,3. low,4. close,5. volume,Close_Prev_Day,Open_Next_Day,Price_Diff
2023-09-28,86.905,90.1100,85.2900,88.96,4128735,91.09,87.06,-4.03
2023-09-27,87.060,88.3600,86.6600,87.83,2256020,88.96,87.93,-1.03
2023-09-26,87.930,88.7500,86.2500,86.57,2835372,87.83,87.84,0.01
2023-09-25,87.840,88.8800,87.4010,88.84,1665995,86.57,89.10,2.53
2023-09-22,89.100,89.7250,87.2600,88.76,2487630,88.84,91.00,2.16


In [51]:
maximum = max(daily_data['Price_Diff'])
daily_data[daily_data['Price_Diff'] == maximum]

,1. open,2. high,3. low,4. close,5. volume,Close_Prev_Day,Open_Next_Day,Price_Diff
2023-08-07,109.22,110.0800,104.7700,106.3,7678806,88.04,114.795,26.755


In [76]:
intraday_ticker_function = "TIME_SERIES_INTRADAY"
interval = "5min"
intraday_ticker_url = f"{base_url}function={intraday_ticker_function}&symbol={symbol}&interval={interval}&apikey={api_key}"
intraday_ticker_response = requests.get(intraday_ticker_url)
intraday_ticker_data = intraday_ticker_response.json()

In [77]:
intraday_data = pd.DataFrame.from_dict(intraday_ticker_data["Time Series (5min)"], orient="index")
intraday_data.head()

,1. open,2. high,3. low,4. close,5. volume
2023-09-29 19:55:00,91.2900,91.2900,91.2800,91.2800,2
2023-09-29 19:20:00,91.4200,91.4200,91.4200,91.4200,2
2023-09-29 19:15:00,91.2100,91.2100,91.2100,91.2100,10
2023-09-29 19:05:00,91.4200,91.4200,91.4100,91.4100,36
2023-09-29 19:00:00,91.4200,91.4200,91.4200,91.4200,1


In [84]:
intraday_data['5. volume'] = intraday_data['5. volume'].astype(int)
intraday_data['volume_change'] = intraday_data['5. volume'] - intraday_data['5. volume'].shift(-1)
intraday_data['avg_volume_change'] = intraday_data[intraday_data['volume_change'] > 0]['volume_change'].shift(-1)
intraday_data.dropna(subset=['volume_change'], inplace=True)
intraday_data.head(6)

,1. open,2. high,3. low,4. close,5. volume,volume_change,avg_volume_change
2023-09-29 19:55:00,91.2900,91.2900,91.2800,91.2800,2,0.0,NaN
2023-09-29 19:20:00,91.4200,91.4200,91.4200,91.4200,2,-8.0,NaN
2023-09-29 19:15:00,91.2100,91.2100,91.2100,91.2100,10,-26.0,NaN
2023-09-29 19:05:00,91.4200,91.4200,91.4100,91.4100,36,35.0,147.0
2023-09-29 19:00:00,91.4200,91.4200,91.4200,91.4200,1,-147.0,NaN
2023-09-29 18:55:00,91.1300,91.1500,91.1000,91.1000,148,147.0,114.0


In [60]:
volume_change_max = max(intraday_data['volume_change'])
intraday_data[intraday_data['volume_change'] == volume_change_max]

,1. open,2. high,3. low,4. close,5. volume,volume_change
2023-09-29 16:00:00,91.0800,91.7500,91.0800,91.0900,807446,807171.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Labeling data (creating a simple binary label: 1 for buy, 0 for not buy)
maximum = max(data[0], data[-1])

# Prepare features and labels
X = data[['SMA_50', 'SMA_200', 'RSI']]
y = data['Buy']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [43]:
# for ticker in tickers:
#     historical_endpoint = "historical-price-full"
#     historical_url = f"{base_url}{historical_endpoint}/{ticker}?apikey={api_key}"
#     historical_response = requests.get(historical_url)
#     historical_data = historical_response.json()

In [18]:
# import concurrent.futures

# def fetch_historical_data(ticker):
#     """Fetches the data from the API and returns it as a JSON response."""
#     historical_url = f"{base_url}{historical_endpoint}/{ticker}?apikey={api_key}"
#     response = requests.get(historical_url)
#     data = response.json()
#     return data

# # This is modified based on the hardware of the computer. 4-8 threads for a cpu is common so I defaulted to 8.
# max_workers = 8

# # Create a ThreadPoolExecutor with max_workers threads
# with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#     # Submit the fetch_historical_data function for each ticker
#     futures = [executor.submit(fetch_historical_data, ticker) for ticker in tickers]

#     # Process the results as they complete
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             data = future.result()
#             # Process the fetched historical data as needed
#             print(data)
#         except Exception as e:
#             print(f"An error occurred: {e}")
